In [17]:
%cd /content
!rm -rf daia-eon   # remove the old clone completely

/content


In [18]:
!git clone https://github.com/AnnaGhost2713/daia-eon.git
%cd daia-eon/data

Cloning into 'daia-eon'...
remote: Enumerating objects: 997, done.
remote: Counting objects: 100% (125/125), done.
remote: Compressing objects: 100% (89/89), done.
remote: Total 997 (delta 49), reused 98 (delta 35), pack-reused 872 (from 1)
Receiving objects: 100% (997/997), 3.44 MiB | 7.39 MiB/s, done.
Resolving deltas: 100% (554/554), done.
/content/daia-eon/data


In [19]:
# --------- LOADING THE DATA FILES ---------

import json
%cd data-eon/data

# Load real E.ON emails
with open("original/original_with_spans.json", encoding="utf-8") as fh:
    real_data = json.load(fh)

# Load synthetic emails
with open("synthetic/synthetic_mails_option_a.json", encoding="utf-8") as fh:
    synthetic_data = json.load(fh)

# Extract only the text
real_texts = [entry["text"] for entry in real_data]
synthetic_texts = [entry["text"] for entry in synthetic_data]

# Preview sample
print("Real email:", real_texts[0][:250])
print("Synthetic email:", synthetic_texts[0][:250])


[Errno 2] No such file or directory: 'data-eon/data'
/content/daia-eon/data
Real email: Wie angefordert der Kontoauszug. Abbuchung Euro 103,22.
Gesendet mit der Stiffel AG Mail App
<http://www.t-online.de/service/redir/emailmobilapp_ios_smartphone_footerlink.htm>

Synthetic email: Hallo liebes Eon Team, es geht um die Vertragsnummer 402 485 275 070. Bei der Errichtung meines neuen Vertrages wurde leider die Banküberweisung von dem jungen Kollegen an der Wohnungstür als Zahlungsmittel gewählt. Ich möchte, dass es wieder per Las


In [20]:
# --------- BERTscore ---------

!pip install bert-score

from bert_score import score
import random

# Sample for evaluation (optional – to keep compute small)
n = min(len(real_texts), len(synthetic_texts), 100)
sample_real = random.sample(real_texts, n)
sample_synthetic = random.sample(synthetic_texts, n)

# Compute BERTScore (synthetic as "candidate", real as "reference")
P, R, F1 = score(cands=sample_synthetic, refs=sample_real, lang="de")

# Print aggregate results
print(f"🔎 BERTScore (German):\nPrecision: {P.mean():.4f}\nRecall: {R.mean():.4f}\nF1: {F1.mean():.4f}")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 110.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 89.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 61.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 104.0 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalli

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

🔎 BERTScore (German):
Precision: 0.6207
Recall: 0.6233
F1: 0.6208


We evaluated the similarity between synthetic emails and real customer communications using BERTScore with a German-language transformer model. The results are:

Precision: 0.6207

Recall: 0.6233

F1 Score: 0.6208

These scores indicate a moderate semantic similarity between the synthetic texts and the real-world emails. BERTScore goes beyond surface-level token overlap and measures how well the meanings of sentences align based on contextual embeddings from a pre-trained transformer model. In this case:

The precision of ~0.62 means that the synthetic content contains a reasonable proportion of semantically relevant tokens compared to the real text.

The recall of ~0.62 reflects that the synthetic emails cover a good portion of the content expressed in real emails.

The F1 score balances both and suggests that the generated texts are semantically aligned but not perfect replicas — which is often ideal when striving for realistic yet privacy-preserving synthetic data.

While not extremely high, a score around 0.62 is acceptable for generation from templates with variable substitution, especially given the diversity of slot values and paraphrasing involved. These results provide a solid baseline for further refinement or comparison with alternative generation approaches.

In [ ]:
# --------- BERTscore ---------
